In [1]:
import pandas as pd


results_df = pd.DataFrame(
    {
        'Model': [],
        'Accuracy': [],
        'Recall': [],
        'ROC-AUC': [],
        'PR-AUC': [],
    }
).astype(
    {
        'Model': str,
        'Accuracy': float,
        'Recall': float,
        'ROC-AUC': float,
        'PR-AUC': float,
    }
)

In [2]:
from module import (
    evaluate_and_append,
    X,
    y,
    skf,
    CAT_FEATURES
)

from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTENC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV


pipeline_knn = ImbPipeline([
    ('smote', SMOTENC(categorical_features=CAT_FEATURES, random_state=42)),
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

param_grid_knn = {
    'smote__k_neighbors': [3, 5, 7],
    'smote__sampling_strategy': [0.5, 0.7, 1.0],
    'knn__n_neighbors': [3, 5, 7, 11, 15, 21, 31, 51],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}

grid_knn = GridSearchCV(
    estimator=pipeline_knn,
    param_grid=param_grid_knn,
    cv=skf,
    scoring='recall',
    n_jobs=-1,
    verbose=1,
)

grid_knn.fit(X, y)

print('KNN best params:', grid_knn.best_params_)
print('KNN best recall:', grid_knn.best_score_)

results_df = evaluate_and_append(
    model_name='KNeighborsClassifier+SMOTENC',
    best_estimator=grid_knn.best_estimator_,
    X=X, y=y, cv=skf,
    results_df=results_df
)

print(results_df)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits
KNN best params: {'knn__metric': 'manhattan', 'knn__n_neighbors': 51, 'knn__weights': 'uniform', 'smote__k_neighbors': 5, 'smote__sampling_strategy': 1.0}
KNN best recall: 0.6046941678520625
                          Model  Accuracy    Recall   ROC-AUC    PR-AUC
0  KNeighborsClassifier+SMOTENC  0.767685  0.604278  0.781483  0.113037
